In [86]:
import spacy
from spacy import displacy
from spacy.training.example import Example
from spacy.training.example import offsets_to_biluo_tags
from langdetect import detect
import pandas as pd
from spacy.util import minibatch, compounding
from unidecode import unidecode

nlp = spacy.load("fr_core_news_lg")


In [87]:
# Les phrases d'entrée
input_data = [
"1,je voudrais aller de Toulouse à bordeaux",
"2,Comment me rendre à Port-Boulet depuis la gare de Tours ?",
"3,Je veux aller voir mon ami Albert à Tours en partant de Bordeaux",
"4,Il y a-t-il des trains de Nantes à Montaigu",
"5,Une phrase sans origine ni destination",
"Si pas de numéro de séquence, on considère que c'est zéro",
"7, is there any train going from Paris to Marseille ?"
]

In [88]:
# Liste des mots pour reconnaître la destination en première
destination_keywords = [
    "à",    
    "vers",
    "jusqu'à",
    "pour",
    "en direction de",
    "jusqu'à",
    "atteindre"
    # ... (ajoutez d'autres expressions si nécessaire)
] 

# Liste des mots pour reconnaître la departure en première
departure_keywords = [
    "depuis",
    "de",
    "au départ de",
    "au",
    "en partant de",
    "depuis",
    "au départ de chez",
    "départ"
    # ... (ajoutez d'autres expressions si nécessaire)
]
# Liste des mots pour reconnaître la departure en première
passage_keywords = [
    "passant par",
    "via",
    "par le chemin de",
    "par la route de",
    "par la voie de",
    "en faisant un détour par",
    "par l'intermédiaire de",
    "en incluant",
    "avec un arrêt à",
    "parmi les étapes à",
    "en traversant",
    "en faisant escale à",
    "tout en visitant",
    "parmi les destinations à",
    "et en découvrant",
    "tout en passant par",
    "tout en explorant",
    "avec un passage à",
    "en chemin vers",
    "en voyage vers",
    # Ajoutez d'autres expressions si nécessaire
]

# Liste des mots a exclure
exclude_words = [
    "gare",
    "Gare",
]

In [89]:
# Fonction pour détecter la langue de la phrase
def detect_language(text):
    try:
        language = detect(text)
        if language != 'fr':   
            return True  # Vérifie si la langue détectée n'est pas le français
    except:
        return False

In [90]:
# Fonction pour extraire les informations pertinentes
def extract_trip_info(text):

    for mot in exclude_words:
        text = text.replace(mot, '')

    doc = nlp(text)

    locs = []
    result = []
    
    is_not_french = detect_language(text)
    if is_not_french:
        locs.append("NOT_FRENCH")
        return locs
 
    for token in doc:

        if token.ent_type_ == "LOC":
            locs.append(token.text)
    print(locs)
    if not locs:  # Si la liste est vide
            result.append('NOT_TRIP')

    # Normalisation des mots-clés des destination
    destination_keywords_normalized = [unidecode(keyword) for keyword in destination_keywords]

    # Normalisation des mots-clés des depart
    departure_keywords_normalized = [unidecode(keyword) for keyword in departure_keywords]

   # Normalisation des mots-clés des passage
    passage_keywords_normalized = [unidecode(keyword) for keyword in passage_keywords]


    departure = None
    destination = None
    passage = None
    departure_count = 0
    destination_count = 0
    passage_count = 0



    for i, token in enumerate(doc):
        # use unicode for text
        text_unicode = token.text
        if token.text in departure_keywords or text_unicode in departure_keywords or token.text in departure_keywords_normalized or text_unicode in departure_keywords_normalized and i < len(doc) - 1:
            if doc[i + 1].text in locs:
                departure = doc[i+1].text
                destination_count += 1  
    for i, token in enumerate(doc):
        # use unicode for text
        text_unicode = unidecode(token.text)
        if token.text in destination_keywords or text_unicode in destination_keywords or token.text in destination_keywords_normalized or text_unicode in destination_keywords_normalized and i > 0:
            if doc[i + 1].text in locs:
                destination = doc[i+1].text
                departure_count += 1

    # for i, token in enumerate(doc):
    #     # use unicode for text
    #     text_unicode = unidecode(token.text)
    #     print(token.text)
    #     if token.text in passage_keywords or text_unicode in passage_keywords or token.text in passage_keywords_normalized or text_unicode in passage_keywords_normalized and i < len(doc) - 1:
    #         if doc[i + 1].text in locs:
    #             passage = doc[i+1].text
    #             passage_count += 1  


    for keyword in passage_keywords:
        keyword_words = keyword.split()  # Divise l'expression-clé en mots
        keyword_length = len(keyword_words)

        for i in range(len(doc) - keyword_length + 1):
            window = [token.text for token in doc[i:i+keyword_length]]

            if window == keyword_words:
                print(window, keyword)
                passage_start = i + keyword_length
                passage_end = passage_start + 1
                if doc[passage_start:passage_end].text in locs:
                    passage = doc[passage_start:passage_end].text
                    passage_count += 1 
                    break

    if departure and destination and departure_count == 1 and destination_count == 1 and passage_count == 0 :
        result = {"DEPARTURE": departure, "DESTINATION": destination}
    elif departure and destination and passage and departure_count == 1 and destination_count == 1 and passage_count == 1:
        result = {"DEPARTURE": departure,"BETWEEN" : passage, "DESTINATION": destination}
    if (departure_count > 1) or (destination_count > 1):
        result = {'NOT_TRIP'}
    elif (departure_count == 0) and (destination_count == 0):
        result = {'NOT_TRIP'}
        
    return result
        

In [95]:


# print(extract_trip_info("Je voudrais aller de toulouse à paris en passant par limoges"))
# print(extract_trip_info("j'aimerais partir chez Albert à Toulon depuis amiens"))
print(extract_trip_info("j'aimerais aller a Paris en partant de Paris"))
# print(extract_trip_info("bonjour, je veux zebi Paris nique Marseille"))

['Paris', 'Paris']
{'DEPARTURE': 'Paris', 'DESTINATION': 'Paris'}


In [101]:
text = "On bouge de Bordeaux pour Lyon ce soir, c'est ok pour toi ?"
text2 = "Y a-t-il un moyen d'atteindre nantes depuis nevers ?"
# text2 = "Y a-t-il un moyen d'atteindre clermont depuis Lille ?"
# text2 = "Trouve-moi un moyen de transport de nimes jusqu'à nevers."
# text2 = "Trouve-moi un itinéraire pour aller de montpellier à pau."
# text2 = "Rendez-vous à dax depuis biaritz."
# text2 = "Recherche le chemin entre colmar et bordeaux."
# text2 = "Je veux prendre un train de lyon à Bordeaux"
# text2 = "Je veux un billet niort, Bourges"
# text2 = "Je veux un billet poitiers Niort"
# text2 = "De mulhouse vers la rochelle"
# text2 = "Je veux faire strasbourg toulon comment puis-je faire ?"
# text2 = "départ de châteauroux vers nantes"
# text2 = "reims annecy"
# text2 = "Rennes Lyon"
# text2 = "Strasbourg paris"
# text2 = "On me demande d'arriver a grenoble depuis tours"
# text2 = "Je veux un billet de train pour paris lyon"
# text2 = "j'aimerais venir a caen en partant de nice"
# text2 = "Je veux me rendre a nimes"
# text2 = "j'aimerais partir pour le havre depuis amiens"
# text2 = "bonjour, je veux partir a rouen depuis marseille"

# print(text.find("Bordeaux"))
# print((text.find("Bordeaux")+ len("Bordeaux")))

# print(text.find("Lyon"))
# print((text.find("Lyon")+ len("Lyon")))
ville_depart = "nevers"
ville_arriver = "nantes"

# print(text2.find(ville_arriver))
# print((text2.find(ville_arriver)+ len(ville_arriver)))
# print(text2.find(ville_depart))
# print((text2.find(ville_depart)+ len(ville_depart)))
data = {"entities": [(text2.find(ville_arriver), (text2.find(ville_arriver)+ len(ville_arriver)), "ARRIVER"), (text2.find(ville_depart), (text2.find(ville_depart)+ len(ville_depart)), "DEPART")]}
# print(data)
print(f'("{text2}", {data})')

("Y a-t-il un moyen d'atteindre nantes depuis nevers ?", {'entities': [(30, 36, 'ARRIVER'), (44, 50, 'DEPART')]})
